In [ ]:
##### install Julia packages #####
PkgList = ["CSV", "DataFrames", "KrylovKit", "StatsBase", "LinearAlgebra", "Statistics", "MultivariateStats", "Distributions", "KernelDensity", "ParallelKMeans", "Clustering", "UMAP", "RCall", "NetworkLayout", "GraphPlot", "Graphs", "Colors", "Compose", "CairoMakie", "Fontconfig", "JLD2"]
using Pkg; for p in PkgList; Pkg.add(p); end


In [ ]:
##### install R packages #####
install.packages(c("BiocManager","ggplot2","RColorBrewer"), repos="https://ftp.yz.yamagata-u.ac.jp/pub/cran")
BiocManager::install(c("clusterProfiler","enrichplot","org.Hs.eg.db"))

In [ ]:
##### SGCRNA #####
using CSV, DataFrames
using RCall
using JLD2
include("../SGCRNA.jl")

## 読込 ##
#  行が遺伝子、列が試料
Data = CSV.read("Result/Norm/normalizedCounts_coding.tsv", header=1, comment="#", delim='\t', DataFrame);
# 相関係数と傾きを計算。fn以降はオプション引数
# fn：指定すると結果を保存する。
# mode：前処理の仕方
#       :NONE -> 初期値。測定誤差を考慮しない。DEGに対して実行する場合はこれ。
#       :LESS -> 標準偏差の分布をプロットしたときにピーク以下を測定誤差として除く。
#       :SIGMA -> 標準偏差の分布をプロットしたときにピークとなる値をσとして2σ以内を測定誤差として除く。
#       :FTEST -> 標準偏差の分布をプロットしたときにピークとなる値をF検定して測定誤差範囲を決める。
# その他のオプション引数に関してはSGCRNA.jl内に記述してあります。
CorData, GradData = CCM(Data.Symbol, Matrix(Data[:,5:end]), fn="Result/coding-FTEST", mode=:FTEST);
# CorData = load_object("Result/coding-FTEST_cor.jld2");
# GradData = load_object("Result/coding-FTEST_grad.jld2");

## スペクトラルクラスタリング ##
# 重要なオプション引数
#       tNodeNum：モジュール内に含まれる最大遺伝子数（初期値：100）この値を超える遺伝子数を有するモジュールのみサブクラスタリングが行われる。
#       depthMax：最大何回サブクラスタリングするか。（初期値：５）
# その他のオプション引数に関してはSGCRNA.jl内に記述してあります。
clust, pos, edge_data = SpectralClustering(CorData, GradData);
# save_object("Result/coding-FTEST_scdata.jld2", (clust, pos, edge_data));
# (clust, pos, edge_data) = load_object("Result/coding-FTEST_scdata.jld2")

## パラメータ設定 ##
# ループで全パラメータを実行してもよい
# d=3が大体ちょうどいい
d = 3; k = maximum(clust[d]);
GeneClust = DataFrame(Gene=names(edge_data), Module=clust[d]);
GeneClust |> CSV.write("Result/GeneCluster_coding.tsv", delim='\t', writeheader=true);

## ネットワーク設定 ##
# 全て描画する場合
nw, new_pos, cnctdf, new_clust, score = SetNetwork(edge_data, clust[d], pos, il=collect(1:k));
# 特定のモジュールだけ描画したい場合
nw, new_pos, cnctdf, new_clust, score = SetNetwork(edge_data, clust[d], pos, il=[2,8]);
# save_object("Result/coding_nwdata.jld2", (nw, new_pos, cnctdf, new_clust, score, edge_data))
# (nw, new_pos, cnctdf, new_clust, score, edge_data) = load_object("Result/coding_nwdata.jld2"); k = maximum(new_clust);

## ノードラベル設定（オプション） ##
# 特定の遺伝子名だけ描画したい場合
NodeLabel = [x in ["BARX1","BARX2","COL14A1","GDF5","MMP1","PRG4","COL10A1","IHH","MMP13","PTH1R","RUNX2","SP7"] ? x : "" for x in names(edge_data)];
# 特定の遺伝子名だけ描画したい場合(特定のモジュールだけ描画する場合)
NodeLabel = unique(sort(vcat(cnctdf.e1,cnctdf.e2)))
NodeLabel = [x in ["BARX1","BARX2","COL14A1","GDF5","MMP1","PRG4","COL10A1","IHH","MMP13","PTH1R","RUNX2","SP7"] ? x : "" for x in NodeLabel]

## ネットワーク描画 ##
# 全てのエッジを描画
DrawNetwork("Result/Fig/bulk_AllNetWork_coding-FTEST.png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, edge_mode=:A)
# エッジスコアの絶対値が0.9以上を描画
DrawNetwork("Result/Fig/bulk_AllNetWork_coding-FTEST-0.9.png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, edge_mode=:A, edge_threshold=0.9)
# エッジスコアが負のみ描画
DrawNetwork("Result/Fig/bulk_NegativeNetWork_coding-FTEST-0.9.png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, edge_mode=:N)
# エッジスコアが正かつエッジスコアの絶対値が0.9以上のみ描画。指定したノードラベルも描画
DrawNetwork("Result/Fig/bulk_PositiveNetWork_coding-FTEST-0.9.png", nw, new_pos, cnctdf, new_clust, k, node_scores=score, node_labels=NodeLabel, edge_mode=:P, edge_threshold=0.9)


# GO解析
for i in 1:k
    commun = names(edge_data)[new_clust .== i]
    R"""
        Pkgs <- c("clusterProfiler", "enrichplot", "org.Hs.eg.db", "ggplot2")
        for(p in Pkgs) suppressMessages(library(p, character.only=T))

        options(ggrepel.max.overlaps = Inf)

        Analysis_GO <- function(x) {
            result <- enrichGO(
                                gene=x,
                                keyType="SYMBOL",
                                OrgDb=org.Hs.eg.db,
                                ont="ALL",                #"BP","CC","MF","ALL"
                                pAdjustMethod="BH",
                                pvalueCutoff=0.05,
                                qvalueCutoff=0.05
                            )
        }
        PlotFig <- function(res) {
            res_simple <- simplify(res)
            Fig <- clusterProfiler::dotplot(res_simple)
            ggsave(paste0("Result/Fig/bulk_GO_cluser-",$i,"_dot_simple.png"), plot=Fig, dpi=400)
            Fig <- clusterProfiler::dotplot(res)
            ggsave(paste0("Result/Fig/bulk_GO_cluser-",$i,"_dot_all.png"), plot=Fig, dpi=400)
        }

        result <- try(Analysis_GO($commun), silent=FALSE)
        if (class(result) != "try-error") {
            y <- try(PlotFig(result), silent=FALSE)
        }
    """
end

